# Importing Libraries

In [1]:
import pandas as pd
import seaborn as sns

# Loading Data

In [ ]:
original_df = pd.read_csv('atp_tennis.csv', header=0)
original_df

# Understanding the Big Picture

In [ ]:
original_df.shape

In [ ]:
original_df.info()

# Data Cleaning

Checking if there are any duplicate rows

In [ ]:
number_of_duplicated_rows = original_df.duplicated().sum()
print(f"There are {number_of_duplicated_rows} duplicated rows in the dataset.")